In [1]:
import os
import datetime
import pandas as pd
from pathlib import Path

In [2]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import FOAF, XSD

Get the path in which the notebook is executing

In [3]:
base_path = str(Path(os.path.abspath(os.getcwd())))

Initialize external ontologies

In [4]:
CNS = Namespace("http://eulersharp.sourceforge.net/2003/03swap/countries#")
F1 = Namespace("http://www.dei.unipd.it/database2/Formula1Ontology#")

# Import circuit data

Create the graph

In [ ]:
g = Graph()
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("f1", F1)

In [36]:
circuits_csv = base_path + '/data/circuits.csv'

circuits_df = pd.read_csv(circuits_csv, sep=',', index_col='circuitId')
circuits_df['alt'].replace(to_replace='\\N',value=None,inplace=True)    #remove '\N' values from the dataset
circuits_df['country'] = circuits_df['country'].str.replace(' ','')     #remove white spaces from country names


,circuitRef,name,location,country,lat,lng,alt,url
circuitId,,,,,,,,
1,albert_park,Albert Park Grand Prix Circuit,Melbourne,Australia,-37.84970,144.96800,10,http://en.wikipedia.org/wiki/Melbourne_Grand_P...
2,sepang,Sepang International Circuit,Kuala Lumpur,Malaysia,2.76083,101.73800,18,http://en.wikipedia.org/wiki/Sepang_Internatio...
3,bahrain,Bahrain International Circuit,Sakhir,Bahrain,26.03250,50.51060,7,http://en.wikipedia.org/wiki/Bahrain_Internati...
4,catalunya,Circuit de Barcelona-Catalunya,Montmeló,Spain,41.57000,2.26111,109,http://en.wikipedia.org/wiki/Circuit_de_Barcel...
5,istanbul,Istanbul Park,Istanbul,Turkey,40.95170,29.40500,130,http://en.wikipedia.org/wiki/Istanbul_Park
...,...,...,...,...,...,...,...,...
75,portimao,Autódromo Internacional do Algarve,Portimão,Portugal,37.22700,-8.62670,108,http://en.wikipedia.org/wiki/Algarve_Internati...
76,mugello,Autodromo Internazionale del Mugello,Mugello,Italy,43.99750,11.37190,255,http://en.wikipedia.org/wiki/Mugello_Circuit
77,jeddah,Jeddah Corniche Circuit,Jeddah,SaudiArabia,21.63190,39.10440,15,http://en.wikipedia.org/wiki/Jeddah_Street_Cir...


Iterate over the circuit's data frame

In [39]:
for index, row in circuits_df.iterrows():
    idC = "circuit"+str(index)  # create a unique identifier for the circuit
    Circuit = URIRef(F1[idC])   # create the circuit

    g.add((Circuit, RDF.type, F1.Circuit))
    g.add((Circuit, F1['name'], Literal(row['name'], datatype=XSD.string)))

    if not row['alt'] == None:
        g.add((Circuit, F1['altitude'], Literal(row['alt'], datatype=XSD.int)))

    Country = URIRef(CNS[row['country']])
    g.add((Circuit, F1['hasCountry'], Country))


Export the serialized graph

In [41]:
with open(base_path + '/rdf/circuits.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

# Import constructor data

Create the graph

In [15]:
g = Graph()
g.bind("foaf", FOAF)
g.bind("xsd", XSD)
g.bind("countries", CNS)
g.bind("f1", F1)

In [16]:
constructors_csv = base_path + '/data/constructors.csv'
constructors_df = pd.read_csv(constructors_csv, sep=',', index_col='constructorId')

Load the CSV files that maps between denominations and country

In [17]:
denom_csv = base_path + '/utils/denom.csv'
denom_df = pd.read_csv(denom_csv, sep=',')
denom_df['country'] = denom_df['country'].str.replace(' ','')     #remove white spaces from country names

Load constructor participation

In [18]:
constructors_df = pd.merge(constructors_df, denom_df, on='nationality', how='inner')

Load constructor's data

In [27]:
constructor_standings_csv = base_path + '/data/constructor_standings.csv'
constructor_standings_df = pd.read_csv(constructor_standings_csv, sep=',', index_col='constructorStandingsId')

In [30]:
constructor_results_csv = base_path + '/data/constructor_results.csv'
constructor_results_df = pd.read_csv(constructor_results_csv, sep=',', index_col='constructorResultsId')

Iterate over the constructor's data frame

In [34]:
for constructorId, row in constructors_df.iterrows():
    idCons = "constructor"+str(constructorId)   #unique identifier for the constructor
    Constructor = URIRef(F1[idCons])

    g.add((Constructor, RDF.type, F1.Constructor))
    g.add((Constructor, F1['name'], Literal(row['name'], datatype=XSD.string)))

    Country = URIRef(CNS[row['country']])
    g.add((Constructor, F1['hasCountry'], Country))

    # Find all the races in which a constructor has participated
    participatedRaces = constructor_results_df.loc[constructor_results_df['constructorId'] == constructorId]

    for participationId, row in participatedRaces.iterrows():
        idPart = "participation"+str(participationId)
        Participate = URIRef(F1[idPart])
        g.add((Participate, RDF.type, F1.Participate))

        #searching the constructor standing data (points, number of victories and position) after the participation 
        g.add((Participate, F1['points_after_race'], Literal(constructor_standings_df['points'].iloc[0], datatype=XSD.int)))
        g.add((Participate, F1['position_after_race'], Literal(constructor_standings_df['position'].iloc[0], datatype=XSD.int)))
        g.add((Participate, F1['number_of_wins'], Literal(constructor_standings_df['wins'].iloc[0], datatype=XSD.int)))

        #add the race_weekend associated to the drive
        idRWE = "raceWeekEnd"+str(row['raceId'])
        g.add((Participate, F1['during'], URIRef(F1[idRWE])))

        #add the drive
        g.add((Constructor, F1['appearIn'], Participate))

Saving data

In [36]:
with open(base_path + '/rdf/constructors.ttl', 'w') as file:
    file.write(g.serialize(format='turtle'))

# Import driver data (still implementing)

In [3]:
base_path = str(Path(os.path.abspath(os.getcwd())))
drivers_csv = base_path + '/data/drivers.csv'    #Read the drivers
races_csv = base_path + '/data/races.csv'        #Read the races
results_csv = base_path + '/data/results.csv'    #Associates drivers and races

In [4]:
driver_df = pd.read_csv(drivers_csv, sep=',', index_col='driverId')
driver_df

,driverRef,number,code,forename,surname,dob,nationality,url
driverId,,,,,,,,
1,hamilton,44,HAM,Lewis,Hamilton,1985-01-07,British,http://en.wikipedia.org/wiki/Lewis_Hamilton
2,heidfeld,\N,HEI,Nick,Heidfeld,1977-05-10,German,http://en.wikipedia.org/wiki/Nick_Heidfeld
3,rosberg,6,ROS,Nico,Rosberg,1985-06-27,German,http://en.wikipedia.org/wiki/Nico_Rosberg
4,alonso,14,ALO,Fernando,Alonso,1981-07-29,Spanish,http://en.wikipedia.org/wiki/Fernando_Alonso
5,kovalainen,\N,KOV,Heikki,Kovalainen,1981-10-19,Finnish,http://en.wikipedia.org/wiki/Heikki_Kovalainen
...,...,...,...,...,...,...,...,...
851,aitken,89,AIT,Jack,Aitken,1995-09-23,British,http://en.wikipedia.org/wiki/Jack_Aitken
852,tsunoda,22,TSU,Yuki,Tsunoda,2000-05-11,Japanese,http://en.wikipedia.org/wiki/Yuki_Tsunoda
853,mazepin,9,MAZ,Nikita,Mazepin,1999-03-02,Russian,http://en.wikipedia.org/wiki/Nikita_Mazepin


In [5]:
races_df = pd.read_csv(races_csv, sep=',', index_col='raceId')
races_df

,year,round,circuitId,name,date,time,url,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
raceId,,,,,,,,,,,,,,,,,
1,2009,1,1,Australian Grand Prix,2009-03-29,06:00:00,http://en.wikipedia.org/wiki/2009_Australian_G...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,2009,2,2,Malaysian Grand Prix,2009-04-05,09:00:00,http://en.wikipedia.org/wiki/2009_Malaysian_Gr...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,2009,3,17,Chinese Grand Prix,2009-04-19,07:00:00,http://en.wikipedia.org/wiki/2009_Chinese_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,2009,4,3,Bahrain Grand Prix,2009-04-26,12:00:00,http://en.wikipedia.org/wiki/2009_Bahrain_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
5,2009,5,4,Spanish Grand Prix,2009-05-10,12:00:00,http://en.wikipedia.org/wiki/2009_Spanish_Gran...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1092,2022,18,22,Japanese Grand Prix,2022-10-09,05:00:00,http://en.wikipedia.org/wiki/2022_Japanese_Gra...,2022-10-07,04:00:00,2022-10-07,08:00:00,2022-10-08,04:00:00,2022-10-08,07:00:00,\N,\N
1093,2022,19,69,United States Grand Prix,2022-10-23,19:00:00,http://en.wikipedia.org/wiki/2022_United_State...,2022-10-21,19:00:00,2022-10-21,22:00:00,2022-10-22,19:00:00,2022-10-22,22:00:00,\N,\N
1094,2022,20,32,Mexico City Grand Prix,2022-10-30,20:00:00,http://en.wikipedia.org/wiki/2022_Mexican_Gran...,2022-10-28,18:00:00,2022-10-28,21:00:00,2022-10-29,17:00:00,2022-10-29,20:00:00,\N,\N


In [6]:
results_df = pd.read_csv(results_csv, sep=',', index_col='resultId')
results_df

,raceId,driverId,constructorId,number,grid,position,positionText,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
resultId,,,,,,,,,,,,,,,,,
1,18,1,1,22,1,1,1,1,10.0,58,1:34:50.616,5690616,39,2,1:27.452,218.300,1
2,18,2,2,3,5,2,2,2,8.0,58,+5.478,5696094,41,3,1:27.739,217.586,1
3,18,3,3,7,7,3,3,3,6.0,58,+8.163,5698779,41,5,1:28.090,216.719,1
4,18,4,4,5,11,4,4,4,5.0,58,+17.181,5707797,58,7,1:28.603,215.464,1
5,18,5,1,23,3,5,5,5,4.0,58,+18.014,5708630,43,1,1:27.418,218.385,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25661,1086,825,210,20,13,16,16,16,0.0,69,\N,\N,37,15,1:23.511,188.856,11
25662,1086,848,3,23,17,17,17,17,0.0,69,\N,\N,43,12,1:23.047,189.911,11
25663,1086,849,3,6,19,18,18,18,0.0,69,\N,\N,60,8,1:22.478,191.221,11


Merge all the CSV on the common columns

In [7]:
merged_df = results_df.merge(driver_df, on="driverId").merge(races_df, on="raceId")
merged_df

,raceId,driverId,constructorId,number_x,grid,position,positionText,positionOrder,points,laps,...,fp1_date,fp1_time,fp2_date,fp2_time,fp3_date,fp3_time,quali_date,quali_time,sprint_date,sprint_time
0,18,1,1,22,1,1,1,1,10.0,58,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
1,18,2,2,3,5,2,2,2,8.0,58,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
2,18,3,3,7,7,3,3,3,6.0,58,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
3,18,4,4,5,11,4,4,4,5.0,58,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
4,18,5,1,23,3,5,5,5,4.0,58,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25655,837,786,51,12,3,2,2,2,6.0,35,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
25656,837,774,154,20,13,\N,R,11,0.0,22,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
25657,837,780,105,30,11,10,10,10,0.0,29,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
25658,837,785,126,26,12,9,9,9,0.0,30,...,\N,\N,\N,\N,\N,\N,\N,\N,\N,\N
